In [8]:
import pandas as pd
import numpy as np
import regex as re
from datetime import datetime
from time import time
from math import ceil
from datetime import timedelta
import unicodedata  

In [35]:
start_time = time()

df_names =[
('df_an', 'AnalíticoNC'),
('df_sn', 'SintéticoNC'),
('df_ic', 'ÍndiceCorreção'),
('df_pr', 'Proponentes'),
('df_cg', 'ControleGeral'),

e1 = 'Controle de NC 2022.xlsx'
e2 = 'Proponentes.xlsx'
e3 = 'Controle de Empenhos e NC 2022.xlsx'   

df_an = pd.read_excel(e1, sheet_name=df_names[0][1],
usecols="B:J", skiprows=1,parse_dates=True)

df_sn = pd.read_excel(e1, sheet_name=df_names[1][1],
usecols="B:J", skiprows=3,parse_dates=True)

df_ic = pd.read_excel(e1, sheet_name=df_names[2][1],
skiprows=1,usecols="B:E")

df_pr = pd.read_excel(e2, sheet_name=0)

df_cg = pd.read_excel(e3, sheet_name=df_names[4][1]
, skiprows=12,usecols="c:w")

##----------------------------------------------------------

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time

/home/roberto/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/roberto/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)


Time: This code took 2 seconds to run.


In [67]:
start_time = time()
# criando função 'regexing' p/ subst. nomes abreviados de forma a normalizar as strings
def regexing(s):
    lst = [
    ('PRF ','POLICIA RODOVIARIA FEDERAL '),
    ('DEP. FEDERAL ', ''),
    ('MCOM ', 'MINISTERIO DAS COMUNICACOES '),
    ('DEP. FEDERAL', ''),
    ('PRF ','POLICIA RODOVIARIA FEDERAL '),
    ('MIN. ','MINISTERIO '),
    ('SENADORA ',''),
    ('SENADOR ',''),
    ]
    for string in lst:
        return re.sub(string[0],string[1],s)



lst_proponente = []

for index, prop in enumerate(df_pr['prop_pk1']):
    if df_pr['orgao'][index] == "senado":
        lst_proponente.append('SEN. ' + prop)
    elif  df_pr['orgao'][index] == "camara":
        lst_proponente.append('DEP. ' + prop)
    else: 
        lst_proponente.append(prop)

df_pr['prop_pk2'] = lst_proponente

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time
#Time: This code took 1 seconds to run.

Time: This code took 1 seconds to run.


In [68]:
start_time = time()

#----------------------------------------------------------------
"""
df_names =[
('df_an', 'AnalíticoNC'),
('df_sn', 'SintéticoNC'),
('df_ic', 'ÍndiceCorreção'),
('df_pr', 'Proponentes')]"""
#put in the same order to make build the dictionary with data names
dfs = [
    df_an,
    df_sn, 
    df_ic, 
    df_pr]
#----------------------------------------------------------------
def strip_accents(cln):
   cln = str(cln)
   return ''.join(c for c in unicodedata.normalize('NFD', cln)if unicodedata.category(c) != 'Mn')

def clean_column_name(cln):
    return strip_accents(cln.strip('[]()').lower().replace(' ','_'))

def rename_df(df):
    new_cln = [clean_column_name(cln) for cln in df.columns.values]
    old_cln = [cln for cln in df.columns.values]
    dct = dict(zip(old_cln, new_cln))
    df.rename(columns=dct,inplace=True)
    lst_df_clns.append(new_cln)
    return df
lst_df_clns = []
for df in dfs:
    rename_df(df)

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time
df_names = list(zip(df_names,lst_df_clns))
df_names

Time: This code took 1 seconds to run.


[(('df_an', 'AnalíticoNC'),
  ['proponente',
   'nota_de_credito',
   'programa',
   'mes',
   'valor_faturado',
   'valor_economizado',
   'valor_utilizado',
   'saldo_no_periodo',
   'relatorio']),
 (('df_sn', 'SintéticoNC'),
  ['nc',
   'proponente',
   'programa',
   'empenho',
   'valor_inicial_nc',
   'valor_utilizado',
   'valor_economizado',
   'saldo',
   'consumo_da_nc_%']),
 (('df_ic', 'ÍndiceCorreção'),
  ['mes', 'indice_de_correcao', 'taxa_selic', 'observacao']),
 (('df_pr', 'Proponente'),
  ['prop_pk1',
   'uf',
   'partido',
   'orgao',
   'prop_1',
   'prop_2',
   'prop_3',
   'prop_4',
   'prop_pk2'])]

In [69]:
start_time = time()

#fazendo o loop sobre a lista acima para aplicar as funções que normalizam a coluna com as chaves dos proponentes
lst_df = [df_sn, df_an]
for df in lst_df: 
    df['proponente'] = df['proponente'].apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
    df['proponente'] = df['proponente'].apply(lambda x:str(x).strip())
#
lst_pr = (df_pr['prop_pk1'], df_pr['prop_1'],df_pr['prop_2'], df_pr['prop_3'],df_pr['prop_4'])
#df_sn['proponente'] = df_sn['proponente'].apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
#df_sn['proponente'] = df_sn['proponente'].apply(lambda x:str(x).strip())
for cln in lst_pr:
    cln = cln.apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
    
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 1 seconds to run.


O objetivo inicial do projeto foi limpar os dados relacionados ao os proponentes com a criação de uma coluna nova referente ao nome civil do proponente para que esses valores pudessem ser usados como chave entre diferentes tabelas que serao parametrizadas entre si.

In [70]:
start_time = time()

#criando lista dos DataFrames pelo qual o loop passara 
dfs = (df_sn, df_an)

#criando as colunas das primary_keys nos 3 dataframes
for df in dfs:
    df['prop_pk1'] = ''
    df['prop_pk2'] = ''
    
#criando lista das colunas referente ao nome dos proponentes 
def filter_key(x):
    return (x == lst_pr[0])|(x==lst_pr[1])|(x==lst_pr[2])|(x==lst_pr[3])

for df in lst_df:
    lst_prop_pk1 = []
    lst_prop_pk2= []
    for index,value in enumerate(df.proponente):
        filt_finding_key =  filter_key(df.proponente.iloc[index])
        result1 = str(df_pr.loc[filt_finding_key].prop_pk1.values).strip("[']")
        result2 = str(df_pr.loc[filt_finding_key].prop_pk2.values).strip("[']")
        lst_prop_pk1.append(result1)
        lst_prop_pk2.append(result2)
    df['prop_pk1'] = lst_prop_pk1
    df['prop_pk2'] = lst_prop_pk2

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 2 seconds to run.


In [71]:
#cleaning analitic sintetic rows

lst = ['SUBTOTAL','TOTAL','NAN']

for value in lst:
    df_sn = df_sn.loc[df_sn['proponente'] != value ]  
    
df_sn.fillna(0,inplace=True)
df_an.fillna(0,inplace=True)

In [72]:
df_sn

,nc,proponente,programa,empenho,valor_inicial_nc,valor_utilizado,valor_economizado,saldo,consumo_da_nc_%,prop_pk1,prop_pk2
0,01/2021,BANCADA DE ALAGOAS,GESAC,2021NE000127,1801472.7,1110000.0,1.441285e+05,691472.7,38.383746,BANCADA-AL,BANCADA-AL
1,02/2021,BANCADA DE ALAGOAS,GESAC,2021NE000156,524400.0,150000.0,4.121571e+04,374400.0,28.604119,BANCADA-AL,BANCADA-AL
2,03/2021,BANCADA DE ALAGOAS,GESAC,2021NE000209,5225812.0,60000.0,5.263889e+05,5165812.0,1.148147,BANCADA-AL,BANCADA-AL
3,04/2021,BANCADA DE MINAS GERAIS,GESAC,2021NE000083,13278264.4,40000.0,1.346667e+06,13238264.4,0.301244,BANCADA-MG,BANCADA-MG
4,05/2021,BANCADA DE SAO PAULO,GESAC,2021NE000171,1544762.0,560000.0,1.069632e+05,984762.0,36.251539,BANCADA-SP,BANCADA-SP
...,...,...,...,...,...,...,...,...,...,...,...
73,73/2021,ULDURICO JUNIOR,WifiBrasil,2021NE000186,141000.0,40000.0,1.005498e+04,101000.0,28.368794,ULDURICO ALENCAR PINTO,DEP. ULDURICO ALENCAR PINTO
74,74/2021,SORAYA THRONICKE,WifiBrasil,2021NE000193,155000.0,20000.0,1.262034e+04,135000.0,12.903226,SORAYA THRONICKE,SEN. SORAYA THRONICKE
75,75/2021,MCOM,WifiBrasil,2021NE000065,500000.0,60000.0,1.041698e+04,440000.0,12.000000,MINISTERIO DAS COMUNICACOES,MINISTERIO DAS COMUNICACOES
76,76/2021,CABO JUNIO AMARAL,WifiBrasil,2021NE000123,93000.0,70000.0,4.498251e+04,23000.0,75.268817,GERALDO JUNIO DO AMARAL,DEP. GERALDO JUNIO DO AMARAL


In [73]:
dfs = [df_an, df_ic]

for index,df in enumerate(dfs): 
    meses = [mes for mes in df.mes.unique()]
    n_mes = ['01/2021','02/2021','03/2021','04/2021','05/2021','06/2021','07/2021','08/2021','09/2021','10/2021','11/2021','12/2021']
    meses = list(zip(meses, n_mes))

    for index,value in enumerate(meses):
        df['mes'].replace(meses[index][0],meses[index][1],inplace=True)


    df['mes']= pd.to_datetime(df['mes'])
    if (df.mes.dtype == '<M8[ns]') == True:
        print(f'The loop is ok!')
        
    else: print('The loops is broke!')
    
#df_an['mes'] = pd.to_datetime(df_an['mes'])

The loop is ok!
The loop is ok!


In [74]:
#checando se tem dfs sem prop_pk 
start_time = time()

dfs  = [df_an, df_sn]
lst_no_cv_name = []
for df in dfs:
    for index, value in enumerate(df.loc[df.prop_pk1==''].proponente.values):
        if value not in lst_no_cv_name:
            lst_no_cv_name.append(df.loc[df.prop_pk1==''].proponente.values[index])
        else:pass

print('DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:')
df_pr_nokey = pd.DataFrame()
df_pr_nokey['prop_nokey']= lst_no_cv_name
df_pr_nokey['prop_pk1']= ''
df_pr_nokey['orgao']=''
df_pr_nokey['uf']=''
df_pr_nokey['partido']=''

save = False
if save == True:
    df_pr_nokey.to_csv('ignore_prop_with_no_key.csv')
else:pass

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time
df_pr_nokey

DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:
Time: This code took 1 seconds to run.


,prop_nokey,prop_pk1,orgao,uf,partido


In [75]:
start_time = time()

#installing the python library to save the data an excel with all the sheets
!pip install xlsxwriter

dfs=[]

#append dfs as str from the dictionary of data to a new list
for i,v in enumerate(df_names):
    dfs.append(df_names[i][0][0]) 

command = str('dfs = ' + str(dfs).replace("'",'')) # building a command with strings

print('This is the command >>>',command)

exec(command) # executing the str as a command

#giving the name to the excel sheets 
for i,df in enumerate(dfs):
  dfs[i].name = df_names[i][0][1]

#creating the excel file
writer = pd.ExcelWriter('BI_Financeiro.xlsx',engine='xlsxwriter')
workbook=writer.book

#putting all dfs inside the excel
for df in dfs:
  worksheet=workbook.add_worksheet(df.name)
  writer.sheets[df.name] = worksheet
  df.to_excel(writer,sheet_name=df.name,startrow=0 , startcol=0, index=False)

#saving
writer.save()

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time
#Time: This code took 1 seconds to run.

Defaulting to user installation because normal site-packages is not writeable
This is the command >>> dfs = [df_an, df_sn, df_ic, df_pr]
Time: This code took 2 seconds to run.


In [76]:
final_time= str(timedelta(seconds= total_time)); print(f'Time: The total time to run all the notebook was {final_time}.')
#Time: The total time to run all the notebook is 00:39 minutes.

Time: The total time to run all the notebook was 0:00:03.385413.
